In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import plotly.graph_objects as go
import plotly.express as px

In [4]:
# Carpeta - data diaria
ruta = "data/raw/osinergmin/"  
archivos = sorted(glob.glob(ruta + "osinergmin_*.csv"))

print(f"Archivos: {len(archivos)}")

dfs = []

for archivo in archivos:
    try:
        df = pd.read_csv(archivo)
        # df["Origen"] = archivo  # para rastrear el archivo
        dfs.append(df)
    except Exception as e:
        print(f"Error leyendo {archivo}: {e}")

# Concatenar todo
osinerg = pd.concat(dfs, ignore_index=True)

osinerg = osinerg.drop_duplicates()

osinerg = osinerg[['Distrito', 'Establecimiento', 'Dirección', 'Teléfono',
                        'Departamento', 'Provincia',
                        'Producto', 'Fecha', 'Precio']]
osinerg.to_excel("data/consolidated/consolidado_combustibles.xlsx", index=False)
osinerg.head(4)


Archivos: 22


,Distrito,Establecimiento,Dirección,Teléfono,Departamento,Provincia,Producto,Fecha,Precio
0,ANCON,CENTER GAS SAC,"PANAMERICANA NORTE KM 44.5, SUB LOTE A-1 ASOCI...",016788937/948581946/981530091,LIMA,LIMA,DB5 S-50 UV,2025-11-01,13.35
1,ANCON,SERVICIOS DE COMBUSTIBLES SANTA FE E.I.R.L.,"AA.HH. LAS PALMERAS MZ. C, LT. 14 (ANTES SERPE...",016983211/960690748/996437408,LIMA,LIMA,DB5 S-50 UV,2025-11-01,13.35
2,ANCON,ESTACIÓN DE SERVICIOS ANCÓN S.A.,"PANAMERICANA NORTE KM. 44, URB. SAN JOSE, MZ. ...",945635871/993487080,LIMA,LIMA,DB5 S-50 UV,2025-11-01,13.60
3,ANCON,EMPRESA DE TRANSPORTES Y SERVICIOS VIRGEN DE L...,AV. LA FLORIDA CDRA. 12 MIRAMAR . LAS CONCHITAS,967743212,LIMA,LIMA,DB5 S-50 UV,2025-11-01,13.79


In [20]:
# Productos a comparar
productos = ["DB5 S-50 UV", "Gasohol Regular", "Gasohol Premium"]

fig = go.Figure()

# Agregamos un histograma para cada producto
for producto in productos:
    fig.add_trace(
        go.Histogram(
            x=osinerg[osinerg["Producto"] == producto]["Precio"],
            name=producto,
            opacity=0.85,
            nbinsx=200
        )
    )

fig.update_xaxes(range=[10, 25])

fig.update_layout(
    title=None,#"Distribución de Precios por Tipo de Combustible",
    xaxis_title="Precio",
    yaxis_title="Frecuencia",
    barmode="overlay",         # superposición
    template="plotly_white",
    width=900,
    height=500,
    legend_title="Producto"
)

fig.show()
fig.write_image("results/distrib_combustibles.pdf")


In [21]:
df = osinerg

df["Fecha"] = pd.to_datetime(df["Fecha"])

# --- Agrupar por día con PROMEDIO GEOMÉTRICO + STD ---
daily = df.groupby("Fecha").agg(
    promedio_geom = ("Precio", lambda x: np.exp(np.log(x).mean())),
    volatilidad   = ("Precio", "std")
).reset_index()

# Renombrar para que todo use el promedio geométrico
daily["precio_base"] = daily["promedio_geom"]

# Bandas geom ± std
daily["banda_sup"] = daily["precio_base"] + daily["volatilidad"]
daily["banda_inf"] = daily["precio_base"] - daily["volatilidad"]

# Orden
daily = daily.sort_values("Fecha").reset_index(drop=True)

# --- Base 100 usando el 29-10-2025 ---
base_fecha = pd.to_datetime("2025-10-29")

base_valor = daily.loc[daily["Fecha"] == base_fecha, "precio_base"]

base_valor = base_valor.iloc[0]

# Índices base 100
daily["idx"]     = 100 * daily["precio_base"] / base_valor
daily["idx_sup"] = 100 * daily["banda_sup"]   / base_valor
daily["idx_inf"] = 100 * daily["banda_inf"]   / base_valor

daily.head()

daily["CATEGORIA"] = "COMBUSTIBLES"

daily.rename(columns={"idx":"VALOR","Fecha":"FECHA"},inplace=True)
daily[['FECHA','CATEGORIA', 'VALOR']].to_excel("data/base100/base_combustibles.xlsx",index=False)

In [45]:
daily[['FECHA','CATEGORIA', 'VALOR']]

,FECHA,CATEGORIA,VALOR
0,2025-10-29,COMBUSTIBLES,100.000000
1,2025-10-30,COMBUSTIBLES,99.949285
2,2025-10-31,COMBUSTIBLES,99.773640
3,2025-11-01,COMBUSTIBLES,99.785960
4,2025-11-02,COMBUSTIBLES,99.782047
5,2025-11-03,COMBUSTIBLES,99.716436
6,2025-11-04,COMBUSTIBLES,99.697225
7,2025-11-05,COMBUSTIBLES,99.982280
8,2025-11-06,COMBUSTIBLES,99.810972
9,2025-11-07,COMBUSTIBLES,99.682094


In [ ]:
fig = go.Figure()

# Línea principal
fig.add_trace(go.Scatter(
    x=daily["FECHA"],
    y=daily["VALOR"],
    mode="lines",
    name="Índice (Base 100)",
    line=dict(width=2)
))


fig.update_layout(
    title=None,#{
    #     'text': "<b>Índice de Combustibles</b><br><span style='font-size:12px;'>Base 100 = 29/10/2025 (Promedio Geométrico)</span>",
    #     'x': 0.5,          # Centrar horizontalmente
    #     'xanchor': 'center',
    #     'yanchor': 'top'
    # },
    xaxis_title="Fecha",
    yaxis_title=None,#"Índice (Base 100)",
    template="plotly_white",
    hovermode="x unified",
    width=900,
    height=500
)

fig.show()
fig.write_image("results/indice_combustibles.pdf")


# ANALISIS EXPLORATORIO - OSINERGMIN

In [61]:
osinerg_1 = osinerg

In [35]:
osinerg_1 = osinerg.copy()

# =======================================================
def remove_outliers(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    # Devuelve SOLO los valores razonables
    return df[(df[col] >= lower) & (df[col] <= upper)]


# --- DESCOMENTA esta línea si deseas eliminar outliers:
osinerg_1 = remove_outliers(osinerg_1, "Precio")
osinerg_1 = remove_outliers(osinerg_1, "Fecha")

osinerg_1 = osinerg_1.dropna(subset=["Precio", "Fecha"])


# =======================================================



In [60]:
# -----------------------------------------
df_time = (
    osinerg_1.groupby(["Fecha", "Producto"])["Precio"]
    .mean()
    .reset_index()
)

fig2 = px.line(
    df_time,
    x="Fecha",
    y="Precio",
    color="Producto",
    title=None,#"Precios Promedio por Producto a lo Largo del Tiempo",
    labels={"Precio": "Precio Promedio (S/)", "Fecha": "Fecha"},
    width=1000, height=550,
    template="plotly_white"
)

fig2.update_layout(
    legend=dict(
        orientation="h",       # horizontal
        yanchor="top",
        y=-0.2,               # mueve la leyenda debajo del gráfico
        xanchor="center",
        x=0.5,               
        bgcolor="rgba(0,0,0,0)" 
    ),
    margin=dict(l=40, r=40, t=40, b=120) 
)

fig2.update_traces(mode="lines+markers")
fig2.show()


In [ ]:
# -----------------------------------------
fig4 = px.box(
    osinerg_1,
    x="Distrito",
    y="Precio",
    color="Producto",
    title=None,#"Distribución de Precios por Producto y Distrito",
    labels={"Precio": "Precio (S/)", "Distrito": "Distrito"},
    width=1400, height=650,
    template="plotly_white"
)

fig4.update_layout(
    legend=dict(
        x=0.98,
        y=0.98,
        xanchor="right",
        yanchor="top",
    ),
    legend_itemclick="toggle",
    legend_itemdoubleclick="toggleothers",
    margin=dict(l=40, r=40, t=40, b=110),
)

fig4.update_layout(xaxis={'categoryorder':'total descending'})
fig4.show()